In [9]:
# dataset
import os
import torch

from starry.utils.config import Configuration
from starry.utils.dataset_factory import loadDataset


torch.set_printoptions(profile="full")

DATA_DIR = os.getenv('DATA_DIR')

config = Configuration.create('configs/paraff-data-test.yaml', volatile=True)
data, = loadDataset(config, data_dir=DATA_DIR, splits='*0/1')

it = iter(data)
next(it)


{'input_ids': tensor([[ 92,  85,  82,  95,  89,  81, 101,   1,  19,  24,  36,   4,   7,  67,
           42,  46,  49,  56,  66,  44,  50,  69,  57,  64,  39,  46,  79,  57,
           40,  79,  57,  65,   3,   5,   7,  39,  46,  49,  57,  68,  66,  44,
           56,  63,   3,   6,   7,  39,  46,  49,  57,  68,  66,  42,  46,  50,
           56,  63,   2,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   

In [20]:
# embeding & linear
import torch.nn as nn
import torch


embed = nn.Embedding(10, 17)
linear = nn.Linear(17, 10)

print(embed.weight.shape, linear.weight.shape)


linear.weight = embed.weight
t1 = (torch.rand(1, 10) * 10).long()
t2 = linear(embed(t1)).argmax(axis=-1)

t1, t2


torch.Size([10, 17]) torch.Size([10, 17])


(tensor([[9, 6, 0, 2, 8, 8, 7, 1, 2, 9]]),
 tensor([[9, 6, 0, 2, 8, 8, 7, 1, 2, 9]]))

In [1]:
# model
import os

from starry.utils.config import Configuration
from starry.utils.dataset_factory import loadDataset
from starry.utils.model_factory import loadModel


DATA_DIR = os.getenv('DATA_DIR')

config = Configuration.create('configs/paraff-test.yaml', volatile=True)
train, val = loadDataset(config, data_dir=DATA_DIR, device='cpu')
model = loadModel(config['model'], postfix='Loss')

it = iter(train)

batch = next(it)
loss, metric = model(batch)

loss, metric


/home/xunan/work/deep-starry-paraff/env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(tensor(4.3895, grad_fn=<NllLoss2DBackward0>), {'acc': 0.0009765625})

In [40]:
import torch


#s = torch.ones(5, 5)
#s[:3, :3] = 0
#torch.triu(s, diagonal=1)


entries = torch.tensor([[1,2,3,0,5,6,7], [1,2,0,4,5,6,7]])

mask = (entries == 0).float()
mask = mask.matmul(torch.triu(torch.ones(7, 7), diagonal=0))
random = torch.rand_like(mask) < 0.4

#(mask, random, (1 - mask) * random)

drops = (1 - mask) * random
print(drops)

indices = torch.arange(7)[None, :].repeat(2, 1) + drops.matmul(torch.triu(torch.ones(7, 7), diagonal=0))
indices = indices.clip(max=6)
indices

tensor([[0., 0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.]])


tensor([[0., 1., 3., 4., 5., 6., 6.],
        [1., 2., 3., 4., 5., 6., 6.]])